# Cleaning eGrid Data

The US Environmental Protection Agency (EPA) releases the [eGrid report](https://www.epa.gov/egrid) each year. This report contains data on each of the 11K power plants in the US and Puerto Rico, including power sources, pollution, and efficiency. It also contains a summary of demographic information for the area surrounding each power plant. 

The data is distributed as an Excel file with multiple sheets for specific portions of the dataset. This project uses data from the 'PLANT' sheet, which summarizes the electricity production and fuel types/emissions from each power plant.

A full description of all terms and data in the dataset can be found in [this guide](https://www.epa.gov/system/files/documents/2023-01/eGRID2021_technical_guide.pdf).

This notebook documents cleaning the data by:

- extracting and renaming the relevant columns
- creating a schema of expected data types to catch irregularities, which led me to:
- casting columns to the correct data types
- removing rows with 'NaN' in critical columns
- normalizing plant owner and utility company names

In [2]:
import pandas as pd
import numpy as np
import pandera as pa
import openpyxl
from fuzzywuzzy import fuzz

## Import Data

The most recent data is from 2021. We will open the plant data (detailing production and emissions for each power plant).

In [3]:
egrid_data = pd.ExcelFile('eGRID2021_data.xlsx')

plant_data = pd.read_excel(egrid_data, 'PLNT21')

plant_data.head()

,Plant file sequence number,Data Year,Plant state abbreviation,Plant name,DOE/EIA ORIS plant or facility code,Plant transmission or distribution system owner name,Plant transmission or distribution system owner ID,Utility name,Utility ID,Plant-level sector,...,Plant wind generation percent (resource mix),Plant solar generation percent (resource mix),Plant geothermal generation percent (resource mix),Plant other fossil generation percent (resource mix),Plant other unknown / purchased fuel generation percent (resource mix),Plant total nonrenewables generation percent (resource mix),Plant total renewables generation percent (resource mix),Plant total nonhydro renewables generation percent (resource mix),Plant total combustion generation percent (resource mix),Plant total noncombustion generation percent (resource mix)
0,SEQPLT,YEAR,PSTATABB,PNAME,ORISPL,OPRNAME,OPRCODE,UTLSRVNM,UTLSRVID,SECTOR,...,PLWIPR,PLSOPR,PLGTPR,PLOFPR,PLOPPR,PLTNPR,PLTRPR,PLTHPR,PLCYPR,PLCNPR
1,1,2021,AK,Agrium Kenai Nitrogen Operations,54452,Homer Electric Assn Inc,19558,Agrium US Inc,179,Industrial CHP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2021,AK,Alakanuk,57053,"Alaska Village Elec Coop, Inc",221,"Alaska Village Elec Coop, Inc",221,Electric Utility,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2021,AK,Allison Creek Hydro,58982,"Copper Valley Elec Assn, Inc",4329,"Copper Valley Elec Assn, Inc",4329,Electric Utility,...,0,0,0,0,0,0,1,0,0,1
4,4,2021,AK,Ambler,60243,"Alaska Village Elec Coop, Inc",221,"Alaska Village Elec Coop, Inc",221,Electric Utility,...,0,0,0,0,0,1,0,0,1,0


Let's extract the columns we need:

### Extract relevant columns

In [4]:
plant_df = plant_data[[
    'Plant file sequence number',
    'Plant state abbreviation',
    'Plant transmission or distribution system owner name',
    'Utility name',
    'Balancing Authority Code',
    'NERC region acronym',
    'eGRID subregion acronym',
    'Plant county name',
    'Plant latitude',
    'Plant longitude',
    'Plant primary fuel',
    'Plant primary fuel category',
    'Plant capacity factor',
    'Plant nameplate capacity (MW)',
    'Plant annual net generation (MWh)',
    'Plant annual CO2 emissions (tons)',
    'Plant annual CO2 equivalent emissions (tons)',
    'Plant annual CO2 total output emission rate (lb/MWh)',
    'Plant annual CO2 equivalent total output emission rate (lb/MWh)',
    'Plant annual coal net generation (MWh)', 
    'Plant annual oil net generation (MWh)', 
    'Plant annual gas net generation (MWh)', 
    'Plant annual nuclear net generation (MWh)', 
    'Plant annual hydro net generation (MWh)', 
    'Plant annual biomass net generation (MWh)',
    'Plant annual wind net generation (MWh)',
    'Plant annual solar net generation (MWh)', 
    'Plant annual geothermal net generation (MWh)',
    'Plant annual other fossil net generation (MWh)', 
    'Plant annual other unknown/ purchased fuel net generation (MWh)',
    'Plant coal generation percent (resource mix)',
    'Plant oil generation percent (resource mix)',
    'Plant gas generation percent (resource mix)',
    'Plant nuclear generation percent (resource mix)',
    'Plant hydro generation percent (resource mix)',
    'Plant biomass generation percent (resource mix)',
    'Plant wind generation percent (resource mix)',
    'Plant solar generation percent (resource mix)',
    'Plant geothermal generation percent (resource mix)',
    'Plant other fossil generation percent (resource mix)',
    'Plant other unknown / purchased fuel generation percent (resource mix)'
]]

## Data cleaning

### Rename columns

The column names are unwieldly and confusing.

In [5]:
plant_df_cols = [
    'plant_sequence_num', # 'Plant file sequence number'
    'state', #'Plant state abbreviation'
    'plant_owner', #'Plant transmission or distribution system owner name'
    'utility_name', #'Utility name'
    'balancing_auth_code', #'Balancing Authority Code'
    'nerc_region', #'NERC region acronym'
    'egrid_subregion', #'eGRID subregion acronym'
    'county', #'Plant county name'
    'latitude', #'Plant latitude'
    'longitude', #'Plant longitude'
    'primary_fuel', #'Plant primary fuel'
    'primary_fuel_category', #'Plant primary fuel category'
    'capacity_factor', #'Plant capacity factor'
    'nameplate_capacity_mw', #'Plant nameplate capacity (MW)'
    'annual_net_generation_mwh', # 'Plant annual net generation (MWh)'
    'annual_co2_emissions_tons', #'Plant annual CO2 emissions (tons)'
    'annual_co2_equiv_emissions_tons', #'Plant annual CO2 equivalent emissions (tons)'
    'annual_co2_emission_rate_lb/mwh', #'Plant annual CO2 total output emission rate (lb/MWh)'
    'annual_co2_equiv_emissions_rate_lb_mwh', #'Plant annual CO2 equivalent total output emission rate (lb/MWh)'
    'annual_coal_net_generation_mwh', #'Plant annual coal net generation (MWh)'
    'annual_oil_net_generation_mwh', #'Plant annual oil net generation (MWh)'
    'annual_gas_net_generation_mwh', #'Plant annual gas net generation (MWh)' 
    'annual_nuclear_net_generation_mwh', #'Plant annual nuclear net generation (MWh)'
    'annual_hydro__net_generation_mwh',#'Plant annual hydro net generation (MWh)'
    'annual_biomass_net_generation_mwh', #'Plant annual biomass net generation (MWh)'
    'annual_wind_net_generation_mwh',#'Plant annual wind net generation (MWh)'
    'annual_solar_net_generation_mwh', #'Plant annual solar net generation (MWh)'
    'annual_geothermal_net_generation_mwh', #'Plant annual geothermal net generation (MWh)'
    'annual_other_fossil_fuel_net_generation_mwh', #'Plant annual other fossil net generation (MWh)'
    'annual_other_purchased_net_generation_mwh',#'Plant annual other unknown/ purchased fuel net generation (MWh)'
    'coal_generation_percent', #'Plant coal generation percent (resource mix)'
    'oil_generation_percent', #'Plant oil generation percent (resource mix)'
    'gas_generation_percent', #'Plant gas generation percent (resource mix)'
    'nuclear_generation_percent', #'Plant nuclear generation percent (resource mix)'
    'hydro_generation_percent', #'Plant hydro generation percent (resource mix)'
    'biomass_generation_percent', #'Plant biomass generation percent (resource mix)'
    'wind_generation_percent', #'Plant wind generation percent (resource mix)'
    'solar_generation_percent', #'Plant solar generation percent (resource mix)'
    'geothermal_generation_percent', #'Plant geothermal generation percent (resource mix)'
    'other_fossil_fuel_generation_percent', #'Plant other fossil generation percent (resource mix)'
    'other_purchased_generation_percent', #'Plant other unknown / purchased fuel generation percent (resource mix)'
]

In [6]:
plant_df.columns = plant_df_cols

In [7]:
plant_df.head()

,plant_sequence_num,state,plant_owner,utility_name,balancing_auth_code,nerc_region,egrid_subregion,county,latitude,longitude,...,oil_generation_percent,gas_generation_percent,nuclear_generation_percent,hydro_generation_percent,biomass_generation_percent,wind_generation_percent,solar_generation_percent,geothermal_generation_percent,other_fossil_fuel_generation_percent,other_purchased_generation_percent
0,SEQPLT,PSTATABB,OPRNAME,UTLSRVNM,BACODE,NERC,SUBRGN,CNTYNAME,LAT,LON,...,PLOLPR,PLGSPR,PLNCPR,PLHYPR,PLBMPR,PLWIPR,PLSOPR,PLGTPR,PLOFPR,PLOPPR
1,1,AK,Homer Electric Assn Inc,Agrium US Inc,NaN,AK,AKGD,Kenai Peninsula,60.6732,-151.3784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AK,"Alaska Village Elec Coop, Inc","Alaska Village Elec Coop, Inc",NaN,AK,AKMS,Wade Hampton,62.6833,-164.6544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,AK,"Copper Valley Elec Assn, Inc","Copper Valley Elec Assn, Inc",NaN,AK,AKMS,Valdez Cordova,61.084444,-146.353333,...,0,0,0,1,0,0,0,0,0,0
4,4,AK,"Alaska Village Elec Coop, Inc","Alaska Village Elec Coop, Inc",NaN,AK,AKMS,Northwest Arctic,67.08798,-157.856719,...,1,0,0,0,0,0,0,0,0,0


Remove first row, these are acronyms for the original column names.

In [8]:
plant_df.drop([0], inplace=True)
plant_df.head()

/var/folders/dv/sn1_d0xn0vncjbncdpkkg_pc0000gn/T/ipykernel_82464/2493680168.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_df.drop([0], inplace=True)


,plant_sequence_num,state,plant_owner,utility_name,balancing_auth_code,nerc_region,egrid_subregion,county,latitude,longitude,...,oil_generation_percent,gas_generation_percent,nuclear_generation_percent,hydro_generation_percent,biomass_generation_percent,wind_generation_percent,solar_generation_percent,geothermal_generation_percent,other_fossil_fuel_generation_percent,other_purchased_generation_percent
1,1,AK,Homer Electric Assn Inc,Agrium US Inc,NaN,AK,AKGD,Kenai Peninsula,60.6732,-151.3784,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AK,"Alaska Village Elec Coop, Inc","Alaska Village Elec Coop, Inc",NaN,AK,AKMS,Wade Hampton,62.6833,-164.6544,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,AK,"Copper Valley Elec Assn, Inc","Copper Valley Elec Assn, Inc",NaN,AK,AKMS,Valdez Cordova,61.084444,-146.353333,...,0,0,0,1,0,0,0,0,0,0
4,4,AK,"Alaska Village Elec Coop, Inc","Alaska Village Elec Coop, Inc",NaN,AK,AKMS,Northwest Arctic,67.08798,-157.856719,...,1,0,0,0,0,0,0,0,0,0
5,5,AK,"Inside Passage Elec Coop, Inc","Inside Passage Elec Coop, Inc",NaN,AK,AKMS,Hoonah-Angoon,57.499166,-134.58614,...,1,0,0,0,0,0,0,0,0,0


Let's run the data through a schema validator to see what other problems we might have.

In [9]:
plant_df_schema = pa.DataFrameSchema({
    'plant_sequence_num': pa.Column(int), 
    'state': pa.Column(str, checks=pa.Check.str_length(min_value=2, max_value=2)),
    'plant_owner': pa.Column(str), 
    'utility_name': pa.Column(str),
    'balancing_auth_code': pa.Column(str), 
    'nerc_region': pa.Column(str), 
    'egrid_subregion': pa.Column(str), 
    'county': pa.Column(str),
    'latitude': pa.Column(float), 
    'longitude': pa.Column(float), 
    'primary_fuel': pa.Column(str, checks=pa.Check(lambda x: x == x.str.upper())), 
    'primary_fuel_category': pa.Column(str, checks=pa.Check(lambda x: x == x.str.upper())),
    'capacity_factor': pa.Column(float), 
    'nameplate_capacity_mw': pa.Column(float), 
    'annual_net_generation_mwh': pa.Column(float),
    'annual_co2_emissions_tons': pa.Column(float),
    'annual_co2_equiv_emissions_tons': pa.Column(float), 
    'annual_co2_emission_rate_lb/mwh': pa.Column(float),
    'annual_co2_equiv_emissions_rate_lb_mwh': pa.Column(float),
    'annual_coal_net_generation_mwh': pa.Column(float), 
    'annual_oil_net_generation_mwh': pa.Column(float),
    'annual_gas_net_generation_mwh': pa.Column(float), 
    'annual_nuclear_net_generation_mwh': pa.Column(float),
    'annual_hydro__net_generation_mwh': pa.Column(float), 
    'annual_biomass_net_generation_mwh': pa.Column(float),
    'annual_wind_net_generation_mwh': pa.Column(float), 
    'annual_solar_net_generation_mwh': pa.Column(float),
    'annual_geothermal_net_generation_mwh': pa.Column(float),
    'annual_other_fossil_fuel_net_generation_mwh': pa.Column(float),
    'annual_other_purchased_net_generation_mwh': pa.Column(float), 
    'coal_generation_percent': pa.Column(float),
    'oil_generation_percent': pa.Column(float), 
    'gas_generation_percent': pa.Column(float),
    'nuclear_generation_percent': pa.Column(float), 
    'hydro_generation_percent': pa.Column(float),
    'biomass_generation_percent': pa.Column(float), 
    'wind_generation_percent': pa.Column(float),
    'solar_generation_percent': pa.Column(float), 
    'geothermal_generation_percent': pa.Column(float),
    'other_fossil_fuel_generation_percent': pa.Column(float),
    'other_purchased_generation_percent': pa.Column(float)
})

Let's try running the schema.

In [10]:
plant_df_schema(plant_df)

SchemaError: expected series 'plant_sequence_num' to have type int64, got object

What's wrong with the plant_sequence_num?

In [11]:
print(plant_df['plant_sequence_num'].dtype)

plant_df['plant_sequence_num'].keys()

object


RangeIndex(start=1, stop=11394, step=1)

The plant sequence column has been set as an index. Let's see what other unexpected column types we have.

In [12]:
plant_df.dtypes

plant_sequence_num                             object
state                                          object
plant_owner                                    object
utility_name                                   object
balancing_auth_code                            object
nerc_region                                    object
egrid_subregion                                object
county                                         object
latitude                                       object
longitude                                      object
primary_fuel                                   object
primary_fuel_category                          object
capacity_factor                                object
nameplate_capacity_mw                          object
annual_net_generation_mwh                      object
annual_co2_emissions_tons                      object
annual_co2_equiv_emissions_tons                object
annual_co2_emission_rate_lb/mwh                object
annual_co2_equiv_emissions_r

## Recast columns as correct types

Switch plant_sequence_num to regular column instead of index.

In [13]:
plant_df['plant_sequence_num'] = list(plant_df['plant_sequence_num'])

plant_df['plant_sequence_num'].dtypes

/var/folders/dv/sn1_d0xn0vncjbncdpkkg_pc0000gn/T/ipykernel_82464/3219294686.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_df['plant_sequence_num'] = list(plant_df['plant_sequence_num'])


dtype('int64')

Cast object columns as integers.

In [14]:
object_cols = [
    'capacity_factor', 
    'nameplate_capacity_mw',
    'latitude',
    'longitude',
    'annual_net_generation_mwh',
    'annual_co2_emissions_tons', 
    'annual_co2_equiv_emissions_tons', 
    'annual_co2_emission_rate_lb/mwh', 
    'annual_co2_equiv_emissions_rate_lb_mwh', 
    'annual_coal_net_generation_mwh', 
    'annual_oil_net_generation_mwh', 
    'annual_gas_net_generation_mwh', 
    'annual_nuclear_net_generation_mwh',
    'annual_hydro__net_generation_mwh',
    'annual_biomass_net_generation_mwh', 
    'annual_wind_net_generation_mwh',
    'annual_solar_net_generation_mwh', 
    'annual_geothermal_net_generation_mwh',
    'annual_other_fossil_fuel_net_generation_mwh', 
    'annual_other_purchased_net_generation_mwh',
    'coal_generation_percent',
    'oil_generation_percent', 
    'gas_generation_percent', 
    'nuclear_generation_percent',
    'hydro_generation_percent', 
    'biomass_generation_percent', 
    'wind_generation_percent', 
    'solar_generation_percent', 
    'geothermal_generation_percent', 
    'other_fossil_fuel_generation_percent', 
    'other_purchased_generation_percent', 
]
object_cols_dict = dict(zip(object_cols, ['float64']*len(object_cols)))

plant_df = plant_df.astype(object_cols_dict)

Rerun the schema

In [15]:
plant_df_schema(plant_df)

SchemaError: non-nullable series 'plant_owner' contains null values:
4396    NaN
9423    NaN
9424    NaN
9425    NaN
9426    NaN
9427    NaN
9428    NaN
9429    NaN
9430    NaN
9431    NaN
9432    NaN
9433    NaN
9434    NaN
9435    NaN
9436    NaN
9437    NaN
9438    NaN
9439    NaN
9440    NaN
9441    NaN
9442    NaN
9443    NaN
9444    NaN
9445    NaN
9446    NaN
9447    NaN
9448    NaN
9449    NaN
9450    NaN
9451    NaN
9452    NaN
9453    NaN
9454    NaN
9455    NaN
9456    NaN
9457    NaN
9458    NaN
9459    NaN
9460    NaN
9461    NaN
9462    NaN
9463    NaN
9464    NaN
9465    NaN
9466    NaN
9467    NaN
9468    NaN
9469    NaN
9470    NaN
9471    NaN
9472    NaN
Name: plant_owner, dtype: object

The plant_owner column has null values. We'll replace the NaN's with "UNKNOWN".

In [16]:
plant_df['plant_owner'] = plant_df['plant_owner'].fillna('UNKNOWN')

In [17]:
plant_df_schema(plant_df)

SchemaError: non-nullable series 'balancing_auth_code' contains null values:
1       NaN
2       NaN
3       NaN
4       NaN
5       NaN
       ... 
9469    NaN
9470    NaN
9471    NaN
9472    NaN
9479    NaN
Name: balancing_auth_code, Length: 227, dtype: object

The balancing_auth_code also has null values, let's replace. 

In [18]:
plant_df['balancing_auth_code'] = plant_df['balancing_auth_code'].fillna('UNKNOWN')

In [19]:
plant_df_schema(plant_df)

SchemaError: non-nullable series 'county' contains null values:
9429    NaN
9430    NaN
9432    NaN
9435    NaN
9442    NaN
9453    NaN
9455    NaN
9458    NaN
9459    NaN
9464    NaN
9467    NaN
9469    NaN
9470    NaN
9471    NaN
9472    NaN
Name: county, dtype: object

Replace NaN's in county column with 'UNKNOWN'. 

In [20]:
plant_df['county'] = plant_df['county'].fillna('UNKNOWN')

Delete any rows with NaN for longitude or latitude, because that means we can't use GeoJSON to find the plant location, if needed

In [21]:
plant_df = plant_df.dropna(subset=['latitude', 'longitude'])

We should also drop any power plant that doesn't have a net generation number.

In [22]:
plant_df = plant_df.dropna(subset=['annual_net_generation_mwh'])

In [23]:
plant_df = plant_df.loc[plant_df['annual_net_generation_mwh'] != 0]

Fill in all NaN fields in the columns for specific fuels.

In [27]:
replace_nan_cols = [
    'capacity_factor',
    'nameplate_capacity_mw',
    'annual_co2_emissions_tons', 
    'annual_co2_equiv_emissions_tons',
    'annual_co2_emission_rate_lb/mwh',
    'annual_co2_equiv_emissions_rate_lb_mwh',
    'coal_generation_percent',
    'oil_generation_percent', 
    'gas_generation_percent',
    'nuclear_generation_percent', 
    'hydro_generation_percent',
    'biomass_generation_percent', 
    'wind_generation_percent',
    'solar_generation_percent', 
    'geothermal_generation_percent',
    'other_fossil_fuel_generation_percent',
    'other_purchased_generation_percent'
]

plant_df[replace_nan_cols] = plant_df[replace_nan_cols].fillna(0)

In [28]:
plant_df.head()

,plant_sequence_num,state,plant_owner,utility_name,balancing_auth_code,nerc_region,egrid_subregion,county,latitude,longitude,...,oil_generation_percent,gas_generation_percent,nuclear_generation_percent,hydro_generation_percent,biomass_generation_percent,wind_generation_percent,solar_generation_percent,geothermal_generation_percent,other_fossil_fuel_generation_percent,other_purchased_generation_percent
3,3,AK,"Copper Valley Elec Assn, Inc","Copper Valley Elec Assn, Inc",UNKNOWN,AK,AKMS,Valdez Cordova,61.084444,-146.353333,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,AK,"Alaska Village Elec Coop, Inc","Alaska Village Elec Coop, Inc",UNKNOWN,AK,AKMS,Northwest Arctic,67.087980,-157.856719,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,AK,"Inside Passage Elec Coop, Inc","Inside Passage Elec Coop, Inc",UNKNOWN,AK,AKMS,Hoonah-Angoon,57.499166,-134.586140,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,AK,Aniak Light & Power Co Inc,Aniak Light & Power Co Inc,UNKNOWN,AK,AKMS,Bethel,61.580678,-159.535643,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,AK,Alaska Electric Light&Power Co,Alaska Electric Light & Power Co.,UNKNOWN,AK,AKMS,Juneau,58.317600,-134.101000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
plant_df_schema(plant_df)

,plant_sequence_num,state,plant_owner,utility_name,balancing_auth_code,nerc_region,egrid_subregion,county,latitude,longitude,...,oil_generation_percent,gas_generation_percent,nuclear_generation_percent,hydro_generation_percent,biomass_generation_percent,wind_generation_percent,solar_generation_percent,geothermal_generation_percent,other_fossil_fuel_generation_percent,other_purchased_generation_percent
3,3,AK,"Copper Valley Elec Assn, Inc","Copper Valley Elec Assn, Inc",UNKNOWN,AK,AKMS,Valdez Cordova,61.084444,-146.353333,...,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,AK,"Alaska Village Elec Coop, Inc","Alaska Village Elec Coop, Inc",UNKNOWN,AK,AKMS,Northwest Arctic,67.087980,-157.856719,...,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,AK,"Inside Passage Elec Coop, Inc","Inside Passage Elec Coop, Inc",UNKNOWN,AK,AKMS,Hoonah-Angoon,57.499166,-134.586140,...,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,AK,Aniak Light & Power Co Inc,Aniak Light & Power Co Inc,UNKNOWN,AK,AKMS,Bethel,61.580678,-159.535643,...,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,AK,Alaska Electric Light&Power Co,Alaska Electric Light & Power Co.,UNKNOWN,AK,AKMS,Juneau,58.317600,-134.101000,...,0.000000,0.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11389,11389,WY,PacifiCorp,NextEra Energy Operating Services LLC,PACE,WECC,NWPP,Uinta,41.287400,-110.555600,...,0.000000,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
11390,11390,WY,Black Hills Power Inc,"Black Hills Power, Inc.",WACM,WECC,RMPA,Campbell,44.285800,-105.383300,...,0.000000,0.002154,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11391,11391,WY,Black Hills Power Inc,"Black Hills Power, Inc.",WACM,WECC,RMPA,Campbell,44.291900,-105.381100,...,0.000000,0.004420,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11392,11392,WY,Black Hills Power Inc,"Black Hills Power, Inc.",WACM,WECC,RMPA,Campbell,44.291900,-105.380600,...,0.000000,0.001662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


The schema validated the dataframe, so our types are correct and we don't have NaN values hanging around.

## Mispellings and Variant Spellings

In [30]:
# check for misspellings in state
plant_df['state'].unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
       'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
       'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
       'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
       'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

'PR' refers to Puerto Rico. State names are OK.

Let's create methods to find potential variant spellings. (Oddly, there is no out-of-the-box solution for this).

In [31]:
def is_potential_variant(str1, str2):
    return fuzz.partial_ratio(str1, str2) > 90

def get_potential_variants(col_name):
    unique_vals = plant_df[col_name].unique()
    potential_variants = []
    for unique_val in unique_vals:
        variant_list = [variant for variant in unique_vals if is_potential_variant(unique_val, variant) and unique_val != variant]
        if variant_list:
            potential_variants.append([unique_val, variant_list])
    return potential_variants

Find alternate spellings of plant owner names.

In [32]:
owner_variants = get_potential_variants('plant_owner')
owner_variants

[['Copper Valley Elec Assn, Inc', ['Valley Electric Assn, Inc']],
 ['WAPA-- Western Area Power Administration',
  ['Western Area Power Administration']],
 ['Pacific Gas & Electric Co', ['Pacific Gas & Electric Co.']],
 ['Pacific Gas & Electric Co.', ['Pacific Gas & Electric Co']],
 ['Anza Electric Coop Inc', ['McKenzie Electric Coop Inc']],
 ['Public Service Co of Colorado',
  ['Public Service Co of NH', 'Public Service Co of NM']],
 ['Northern States Power Co - Minnesota', ['Northern States Power Co']],
 ['Delmarva Power', ['Delmarva Power & Light Company']],
 ['3 Phases Renewables Inc', ['3 Phases Renewables']],
 ['Florida Keys El Coop Assn, Inc', ['West Florida El Coop Assn, Inc']],
 ['Dominion Energy South Carolina, Inc', ['Dominion Energy South Carolina']],
 ['Tri-County Elec Member Corp', ['Four County Elec Member Corp']],
 ['Farmers Electric Coop, Inc - (IA)', ['Farmers Electric Coop, Inc - (NM)']],
 ['American Transmission Systems Inc', ['American Transmission Co']],
 ['City of

In [33]:
owner_name_variants = [
    {
        'name': 'Pacific Gas & Electric Co.',
        'variants': ['Pacific Gas & Electric Co']
    },
    {
        'name': 'West Penn Power Company',
        'variants': ['West Penn Power Co']
    },
    {
        'name': 'Dominion Energy South Carolina',
        'variants': ['Dominion Energy South Carolina, Inc']
    }
]

In [34]:
def replace_variant_names(col, variant_list):
    for variant in variant_list:
        plant_df.loc[plant_df[col].isin(variant['variants']), col] = variant['name']

In [35]:
replace_variant_names('plant_owner', owner_name_variants)

In [36]:
# This name was replaced, so we expect an empty dataframe

plant_df[plant_df['plant_owner'] == 'West Penn Power Co']

,plant_sequence_num,state,plant_owner,utility_name,balancing_auth_code,nerc_region,egrid_subregion,county,latitude,longitude,...,oil_generation_percent,gas_generation_percent,nuclear_generation_percent,hydro_generation_percent,biomass_generation_percent,wind_generation_percent,solar_generation_percent,geothermal_generation_percent,other_fossil_fuel_generation_percent,other_purchased_generation_percent


Let's do the same process for utility names.

In [37]:
get_potential_variants('utility_name')

[['U S Air Force-Eielson AFB', ['US Air Force']],
 ['Alabama Municipal Elec Authority', ['Municipal Electric Authority']],
 ['International Paper Co.',
  ['International Paper Co',
   'International Paper Co-Riverdl',
   'International Paper Co-Pensacola',
   'International Paper Port Wentworth Mill',
   'International Paper Columbus Mill',
   'International Paper Co-Vicksbg',
   'International Paper Co-Riegelwood',
   'International Paper Co. - New Bern Mill',
   'International Paper Corporation - Springfield',
   'International Paper Co-GT Mill',
   'International Paper']],
 ['International Paper Co',
  ['International Paper Co.',
   'International Paper Co-Riverdl',
   'International Paper Co-Pensacola',
   'International Paper - Flint River Mill',
   'International Paper Port Wentworth Mill',
   'International Paper Columbus Mill',
   'International Paper Co-Vicksbg',
   'International Paper Co-Riegelwood',
   'International Paper Co. - New Bern Mill',
   'International Paper - Val

In [38]:
utility_name_variants = [{'name': 'International Paper Co.',
  'variants': ['International Paper Co',
   'International Paper Co-Riverdl',
   'International Paper Co-Pensacola',
   'International Paper Port Wentworth Mill',
   'International Paper Columbus Mill',
   'International Paper Co-Vicksbg',
   'International Paper Co-Riegelwood',
   'International Paper Co. - New Bern Mill',
   'International Paper Corporation - Springfield',
   'International Paper Co-GT Mill',
   'International Paper']},
 {'name': 'NRG Energy Services',
  'variants': ['NRG Energy Services - Plum Point']},
 {'name': 'Renewable Energy LLC',
  'variants': ['WM Renewable Energy LLC',
   'Hanford Renewable Energy LLC',
   'Plainfield Renewable Energy, LLC',
   'WM Illinois Renewable Energy LLC',
   'Cadillac Renewable Energy LLC',
   'Black Creek Renewable Energy LLC',
   'Uwharrie Mountain Renewable Energy, LLC',
   'Republic Services Renewable Energy, LLC',
   'Covanta Tulsa Renewable Energy LLC',
   'Richland Center Renewable Energy LLC']},
 {'name': 'University of Arizona',
  'variants': ['University of Arizona - Biosphere 2']},
 {'name': 'ABEC LLC',
  'variants': ['ABEC #2 LLC', 'ABEC #3 LLC', 'ABEC #4 LLC']},
 {'name': 'Ameresco',
  'variants': ['Ameresco Butte County LLC',
   'Ameresco Forward, LLC',
   'Ameresco Johnson Canyon LLC',
   'Ameresco San Joaquin, LLC',
   'AMERESCO Santa Cruz Energy LLC',
   'Ameresco Vasco Road, LLC']},
 {'name': 'Los Angeles County', 'variants': ['Los Angeles County Sanitation']},
 {'name': 'Chevron USA Inc', 'variants': ['Chevron USA Inc-El Segundo']},
 {'name': 'Frito-Lay Inc', 'variants': ['Frito Lay Incorporated Dayville']},
 {'name': 'Phillips 66', 'variants': ['Phillips 66 - Los Angeles']},
 {'name': 'Roseburg Forest Products',
  'variants': ['Roseburg Forest Products Co']},
 {'name': 'Valero Refining Co', 'variants': ['Valero Refining Co California']},
 {'name': 'Western Sugar Cooperative',
  'variants': ['Western Sugar Cooperative  - Billings']},
 {'name': 'Kimberly-Clark Corporation', 'variants': ['Kimberly-Clark Corp']},
 {'name': 'Ascend Performance Materials',
  'variants': ['Ascend Performance Materials LLC']},
 {'name': 'Landfill Energy Systems',
  'variants': ['Landfill Energy Systems Florida']},
 {'name': 'WestRock Corp', 'variants': ['WestRock Corp-Panama City']},
 {'name': 'SEPG Operating Services, LLC ALS',
  'variants': ['SEPG Operating Services, LLC MPC',
   'SEPG Operating Services, LLC Walton',
   'SEPG Operating Services, LLC WCP']},
 {'name': 'Graphic Packaging International',
  'variants': ['Graphic Packaging International - Augusta']},
 {'name': 'Cargill Inc', 'variants': ['Cargill Inc North America Sweetners']},
 {'name': 'BASF Corporation', 'variants': ['BASF Corp']},
 {'name': 'Dow Chemical Co',
  'variants': ['Dow Chemical Co - St Charles',
   'Dow Chemical Company-Oyster Creek VIII']},
 {'name': 'Graphic Packaging International',
  'variants': ['Graphic Packaging International - Augusta',
   'Graphic Packaging International - Texarkana']},
 {'name': 'Shell Chemical LP', 'variants': ['Shell Chemical LP - Geismar']},
 {'name': 'Covanta Energy Co', 'variants': ['Covanta Energy of Niagara LP']},
 {'name': 'Smithfield Packaged Meats Corp.',
  'variants': ['Smithfield Packaged Meats Corp']},
 {'name': 'Pixelle Specialty Solution, LLC',
  'variants': ['Pixelle Specialty Solutions LLC - (PA)']},
 {'name': 'Georgia-Pacific Consumer Operations LLC',
  'variants': ['Georgia-Pacific Consumer Operations LLC-Wauna']},
 {'name': 'Vitro Architectural Glass',
  'variants': ['Vitro Architectural Glass (PA)']},
 {'name': 'Ahlstrom-Munksjo',
  'variants': ['Ahlstrom-Munksjo NA Specialty Solutions, LLC']}]

In [39]:
replace_variant_names('utility_name', utility_name_variants)

## Export to CSV

In [40]:
plant_df.to_csv('cleaned_egrid_data.csv')